In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk

In [3]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/anastasiia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords

In [5]:
from pymystem3 import Mystem

In [6]:
from string import punctuation

In [7]:
import re

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

In [9]:
import fasttext

In [10]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### Load the data

In [11]:
train = pd.read_csv('data/train_data.csv')
test = pd.read_csv('data/test_data.csv')

In [12]:
train.head()

,Пример текста,Класс
0,хочу в отпуск,VACATION-REQUEST
1,мне бы в отдохнуть,VACATION-REQUEST
2,как мне взять отпуск,VACATION-REQUEST
3,хочу отгул на следующей неделе,VACATION-REQUEST
4,хочу улететь в турцию,VACATION-REQUEST


In [13]:
train.columns = ['message', 'label']
test.columns = ['message', 'label']

In [14]:
train.head()

,message,label
0,хочу в отпуск,VACATION-REQUEST
1,мне бы в отдохнуть,VACATION-REQUEST
2,как мне взять отпуск,VACATION-REQUEST
3,хочу отгул на следующей неделе,VACATION-REQUEST
4,хочу улететь в турцию,VACATION-REQUEST


In [15]:
train['label'].value_counts()

SICK-LEAVE-REPORT    11
SALARY-REQUEST       11
VACATION-REQUEST     10
Name: label, dtype: int64

In [16]:
train

,message,label
0,хочу в отпуск,VACATION-REQUEST
1,мне бы в отдохнуть,VACATION-REQUEST
2,как мне взять отпуск,VACATION-REQUEST
3,хочу отгул на следующей неделе,VACATION-REQUEST
4,хочу улететь в турцию,VACATION-REQUEST
5,нужно два дня отпуска на следующей неделе,VACATION-REQUEST
6,уйду в отпуск в октябре,VACATION-REQUEST
7,можно я отдохну,VACATION-REQUEST
8,мне нужно взять пару дней выходных,VACATION-REQUEST
9,я очень хочу отдохнуть,VACATION-REQUEST


In [17]:
test

,message,label
0,Я хочу сходить в отпуск с 1 по 4 декабря,VACATION-REQUEST
1,Мне нужен отпуск с понедельника,VACATION-REQUEST
2,Как мне получить оптуск,VACATION-REQUEST
3,Я буду отдыхать на следующей неделе,VACATION-REQUEST
4,Какая у меня была зарплата за прошлый месяц?,SALARY-REQUEST
5,Сколько я заработал в январе?,SALARY-REQUEST
6,Пришлите мне рассетный лист,SALARY-REQUEST
7,Сколько всего мне зраплатят в апреле?,SALARY-REQUEST
8,Мне нужно взять больничный на пару дней,SICK-LEAVE-REPORT
9,Я хочу сегодня сходить к доктору,SICK-LEAVE-REPORT


### Preprocessing

In [18]:
mystem = Mystem() 
stopwords = stopwords.words("russian")

In [19]:
def preprocess_message(message):
    #lemmatize
    tokens = mystem.lemmatize(message.lower())
    #remove stopwords
    tokens = [token for token in tokens if token not in stopwords
              and token != " " 
              and token.strip() not in punctuation]
    
    result = " ".join(tokens)
    
    #remove numbers from the string
    result = re.sub(r'\d+', '', result)
    
    return result

In [20]:
train['message_processed'] = train['message'].apply(preprocess_message)

In [21]:
train.head()

,message,label,message_processed
0,хочу в отпуск,VACATION-REQUEST,хотеть отпуск
1,мне бы в отдохнуть,VACATION-REQUEST,отдыхать
2,как мне взять отпуск,VACATION-REQUEST,взять отпуск
3,хочу отгул на следующей неделе,VACATION-REQUEST,хотеть отгул следующий неделя
4,хочу улететь в турцию,VACATION-REQUEST,хотеть улетать турция


In [22]:
test['message_processed'] = test['message'].apply(preprocess_message)

### Model training

In [23]:
#encode labels 
encoder = LabelEncoder()

train_y = encoder.fit_transform(train['label'])
test_y = encoder.transform(test['label'])

train['label_encoded'] = train_y
test['label_encoded'] = test_y

In [24]:
train.head()

,message,label,message_processed,label_encoded
0,хочу в отпуск,VACATION-REQUEST,хотеть отпуск,2
1,мне бы в отдохнуть,VACATION-REQUEST,отдыхать,2
2,как мне взять отпуск,VACATION-REQUEST,взять отпуск,2
3,хочу отгул на следующей неделе,VACATION-REQUEST,хотеть отгул следующий неделя,2
4,хочу улететь в турцию,VACATION-REQUEST,хотеть улетать турция,2


In [25]:
test

,message,label,message_processed,label_encoded
0,Я хочу сходить в отпуск с 1 по 4 декабря,VACATION-REQUEST,хотеть сходить отпуск декабрь,2
1,Мне нужен отпуск с понедельника,VACATION-REQUEST,нужный отпуск понедельник,2
2,Как мне получить оптуск,VACATION-REQUEST,получать оптускнуть,2
3,Я буду отдыхать на следующей неделе,VACATION-REQUEST,отдыхать следующий неделя,2
4,Какая у меня была зарплата за прошлый месяц?,SALARY-REQUEST,зарплата прошлый месяц,0
5,Сколько я заработал в январе?,SALARY-REQUEST,сколько зарабатывать январь,0
6,Пришлите мне рассетный лист,SALARY-REQUEST,присылать рассетный лист,0
7,Сколько всего мне зраплатят в апреле?,SALARY-REQUEST,сколько зраплатить апрель,0
8,Мне нужно взять больничный на пару дней,SICK-LEAVE-REPORT,нужно взять больничный пара день,1
9,Я хочу сегодня сходить к доктору,SICK-LEAVE-REPORT,хотеть сегодня сходить доктор,1


In [26]:
#make training data file
train['target'] = train['label'].apply(lambda x: '__label__' + str(x))
train[['target', 'message_processed']].to_csv('data/train_data.txt', header=False, index=False, sep="\t")

In [53]:
#train fasttext classifier
model = fasttext.train_supervised(input='data/train_data.txt', epoch=20)

In [54]:
#save trained model
model.save_model('trained_model/model_20ep.sav')

In [55]:
#load trained model
model = fasttext.load_model('trained_model/model_20ep.sav')

In [60]:
#make predictions on the test data
preds = test['message_processed'].apply(model.predict)

In [61]:
pred_labels = [p[0][0].split('__label__')[1] for p in preds.tolist()]

In [62]:
pred_labels

['VACATION-REQUEST',
 'VACATION-REQUEST',
 'SICK-LEAVE-REPORT',
 'VACATION-REQUEST',
 'SALARY-REQUEST',
 'SALARY-REQUEST',
 'SALARY-REQUEST',
 'SALARY-REQUEST',
 'SICK-LEAVE-REPORT',
 'SICK-LEAVE-REPORT',
 'SICK-LEAVE-REPORT',
 'SICK-LEAVE-REPORT']

In [63]:
#encode predicitons
pred_encoded = encoder.transform(pred_labels)
pred_encoded

array([2, 2, 1, 2, 0, 0, 0, 0, 1, 1, 1, 1])

In [64]:
#check f1 score on the test set
f1_score(test_y, pred_encoded, average='weighted')

0.9153439153439153

In [65]:
#check accuracy on the test set
accuracy_score(test_y, pred_encoded)

0.9166666666666666